In [18]:
# Problem Parameters

max_rooms_per_course = 3 # A course can be alloted to at most 3 rooms
max_courses_per_student_per_day = 2 # A student can have at most 2 courses in a day
weight_minimize_two_exams_per_day = 1.0 # Weight allocated to minimize no. of 2 exams in a day for a student
weight_minimize_rooms_used = 4.0  # Weight allocated to minimize no. of rooms used for a course
weight_professor_preferred_slots = 6.0  # Weight allocated to get preffered slots for professors


In [1]:
! pip install tabulate
! pip install matplotlib
! pip install reportlab
! pip install pypdf2


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex
from openpyxl import Workbook
import import_ipynb
import data_utils as data
import time
import pandas as pd
from collections import defaultdict
from tabulate import tabulate
from reportlab.lib import colors
from reportlab.lib.pagesizes import LETTER
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import PyPDF2
import os 


tt = gp.Model('IIITB Exams Timetable')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2615624
Academic license 2615624 - for non-commercial use only - registered to mo___@iiitb.ac.in


In [3]:
def exam_slots(course: int):
    return int(data.get_course_duration(course)//15)

exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c, r)
        for c in range(data.n_courses)
        for r in data.group_by_cat[data.course_group[c]] 
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="C",
)

M = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in data.group_by_cat[data.course_group[c]]
    ],
    vtype=GRB.BINARY,
    name = 'M',
)

tt.addConstrs(
    ((M[c, r]==1) >> (C[c, r] >= 1)
    for c in range(data.n_courses)
    for r in data.group_by_cat[data.course_group[c]]),
    name = 'max_rooms_per_course1'
)

tt.addConstrs(
    ((M[c, r]==0) >> (C[c, r]==0)
    for c in range(data.n_courses)
    for r in data.group_by_cat[data.course_group[c]]),
    name = 'max_rooms_per_course2'
)

tt.update()


## Hard Constraints

In [4]:
def signum(x):
    if x>0:
        return 1
    return 0

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if  data.overlap(b,e,t)]

In [5]:
# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)

tt.update()

In [6]:
# Constraint 2: All students for a course should write the exam

tt.addConstrs(
    # gp.quicksum(C[c,r] for r in range(data.n_rooms)) == data.get_course_strength(c)
    # for c in range(data.n_courses)\
    gp.quicksum(C[c,r] for r in data.group_by_cat[data.course_group[c]]) == data.get_course_strength(c)
    for c in range(data.n_courses)
)

tt.update()


In [7]:
# Constraint 3: Student clash constraint

tt.addConstrs((gp.quicksum(E[c, b, e, d] for c in data.intersection_pairs[i] for b, e in covering_sessions(c, t)) <= 1
               for i in range(data.n_intersection_pairs)
               for d in range(data.n_days)
               for t in range(data.n_times)),
              name='student_clash_constraint'
             )

tt.update()

In [8]:
# # Constraint 4: No course can have more than 3 rooms alloted

# tt.addConstrs(
#     (gp.quicksum(M[c,r] for r in range(data.n_rooms)) <= max_rooms_per_course
#     for c in range(data.n_courses))
# )

# tt.update()


In [9]:
# Make L[c, b, e, d, r] as product of E[c, b, e, d] and C[c, r] and L_ind as product of E[c, b, e, d] and M[c, r]

L_ind = tt.addVars(
    [
        (c, b, e, d, r)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
        for r in data.group_by_cat[data.course_group[c]]
    ],
    vtype=GRB.BINARY,
    name="L_ind",
)

L = tt.addVars(
    [
        (c, b, e, d, r)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
        for r in data.group_by_cat[data.course_group[c]]
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="L",
)

for c in range(data.n_courses):
    for b, e in exam_spans[c]:
        for d in range(data.n_days):
            for r in data.group_by_cat[data.course_group[c]]:
                tt.addConstr(L_ind[c, b, e, d, r] == gp.and_(E[c, b, e, d], M[c, r]), name=f'L_ind_{c}_{b}_{e}_{d}_{r}')
                tt.addConstr((L_ind[c, b, e, d, r]==1) >> (L[c, b, e, d, r]==C[c, r]), name=f'L_{c}_{b}_{e}_{d}_{r}')
                tt.addConstr((L_ind[c, b, e, d, r]==0) >> (L[c, b, e, d, r]==0), name=f'L_{c}_{b}_{e}_{d}_{r}') 


tt.update()


In [10]:
# Constraint 5: Never exceed room capacity and two courses dont get alloted back to back in the room

for d in range(data.n_days):
    for r in range(data.n_rooms):
        for t in range(data.n_times):
            tt.addConstr(gp.quicksum((L[c, b, e, d, r]
                          for c in data.courses_in_room[r]
                          for b,e in covering_sessions(c, t))) <= data.get_room_capacity(r), 
                          name=f'room_capacity_{d}_{r}_{t}')
            # tt.addConstr(gp.quicksum((L_ind[c, b, e, d, r]
            #               for c in range(data.n_courses)
            #               for b,e in covering_sessions(c, t))) <= 1, 
            #               name=f'room_clash_{d}_{r}_{t}')    
tt.update()

In [11]:
# Constraint 6: Lab exams are conducted in labs

# .addConstrs(
#     (
#         gp.quicksum(M[c, r] for r in (data.get_not_rooms_course(c)))== 0
#         for c in range(data.n_courses)
#     ),
#     name="lab_exam_constraint",
# )

# tt.update()

In [ ]:
# Constraint 7: No triplet courses should be scheduled in the day

tt.addConstrs(
    (
        (E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) + E.sum(c3, '*', '*', d) <= max_courses_per_student_per_day)
        for c1, c2, c3 in data.intersection_triplets
        for d in range(data.n_days)
    ),
    name="triplet_constraint",
)


## Soft Constraints

In [13]:
# # 1. No 2 exams on a day for a student

# # minimize summation courseintersection(c1, c2) * ((E[c1, *, *, d] + E[c2, *, *, d]) > 1) for all c1, c2, d



X = tt.addVars(
    [
        (c1, c2, d)
        for c1, c2 in data.intersection_pairs
        for d in range(data.n_days)
    ],
    vtype = GRB.BINARY, 
    name = 'X'
)

for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        tt.addConstr(E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) - X[c1, c2, d] <= 1 ,
                        name = 'soft_constraint1'
                    )
        

tt.update()

## Objective Function

In [14]:
# obj is summation of X[c1, c2, d] * weight(c1, c2) for all c1, c2, d
# for c1, c2 in data.intersection_pairs:
#     for d in range(data.n_days):
#         obj1 += X[c1, c2, d] 
obj1 = gp.quicksum(X[c1, c2, d]*(len(data.get_intersection_of_courses(c1, c2))) for c1, c2 in data.intersection_pairs for d in range(data.n_days))



In [15]:
# Soft Constraint 2: minimize the number of rooms used
obj2 = gp.quicksum(M[c,r] for c in range(data.n_courses) for r in data.group_by_cat[data.course_group[c]])


In [16]:
# obj3 = gp.quicksum(E[c, b, e, d] for c in range(data.n_courses) for b, e, d in data.get_preferred_slots(c))
# for c in range(data.n_courses):
#     for b, e, d in data.get_preferred_slots(c):
#         print("c: ", c, "b: ", b, "e: ", e, "d: ", d)
#         obj3 += E[c, b, e, d]

#add veriable Z[c] for each course c. Z[c] = 1 if the course is scheduled in the preferred slot
Z = tt.addVars(
    [
        (c)
        for c in range(data.n_courses)
    ],
    vtype=GRB.BINARY,
    name="Z",
)

tt.addConstrs(
    (Z[c] == gp.quicksum((E[c, b, e, d] for b, e, d in data.get_preferred_slots(c)))
    for c in range(data.n_courses)), 
    name="soft_constraint3"
)

tt.update()

obj3 = gp.quicksum(Z[c] for c in range(data.n_courses))

In [19]:
objective =  obj2*weight_minimize_rooms_used - obj3*weight_professor_preferred_slots 
tt.setObjective(objective, sense=GRB.MINIMIZE)
tt.update()

In [20]:



# def cb(model, where):
#     if where == GRB.Callback.MIPNODE:
#         # Get model objective
#         obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

#         # Has objective changed?
#         if abs(obj - model._cur_obj) > 1e-8:
#             # If so, update incumbent and time
#             model._cur_obj = obj
#             model._time = time.time()

#     # Terminate if objective has not improved in 60s
#     if time.time() - model._time > 100:
#         model.terminate()

# tt._cur_obj = float('inf')
# tt._time = time.time()

# tt.optimize(callback=cb)
# # # put time limit of 1200 seconds

# # run it for 180 sec and dual gap 5 %

tt.setParam('MIPGap', 0.25)
tt.setParam('TimeLimit', 300)
tt.optimize()

Set parameter MIPGap to value 0.25
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  300
MIPGap  0.25

Academic license 2615624 - for non-commercial use only - registered to mo___@iiitb.ac.in
Optimize a model with 84699 rows, 319729 columns and 2479737 nonzeros
Model fingerprint: 0xd6568fd9
Model has 465896 simple general constraints
  154320 AND, 311576 INDICATOR
Variable types: 0 continuous, 319729 integer (163941 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 6e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 74418 rows and 2032 columns (presolve time = 5s)...
Presolve

In [ ]:


def clash_report(X_sol, wb):
    wb.append(["Course 1", "Course 2", "Clash"])  
    for (c1, c2) in data.intersection_pairs:
        clash = 0
        for d in range(data.n_days):
            clash += X_sol[c1, c2, d]
        print(c1, c2, clash)
        wb.append([data.course_id_code[c1], data.course_id_code[c2], "True" if clash>=1 else "False"])
        

def exam_slot_report(Esol, Z_sol, wb):
    wb.append(["Course Code", "Title", "size", "CSE/ECE", "day", "duration", "begin", "end", "preferred_time_slot"])

    for c, b, e, d in Esol:
        wb.append([data.course_id_code[c], data.get_course_title(c), data.get_course_strength(c), data.get_dept(c), data.days[d], data.get_course_duration(c), data.times[b], data.times[e], "True" if Z_sol[c] else "False"])

def seating_report(C_sol, Csol, wb):
    wb.append(["Course Code", "Title", "Room", "Room Type", "size", "Room Number", "Capacity", "Seating"])

    for c, r in Csol:
        wb.append([data.course_id_code[c], data.get_course_title(c), data.get_room_num(r), data.get_course_category(c), data.get_course_strength(c), r, data.get_room_capacity(r), C_sol[c, r]])

def make_report(s):
    C_sol = s['C_sol']
    E_sol = s['E_sol']
    M_sol = s['M_sol']
    X_sol = s['X_sol']
    Z_sol = s['Z_sol']
    Esol = []
    for i in E_sol:
        if E_sol[i] > 0:
            Esol.append(i)
    Csol = []
    for i in C_sol:
        if C_sol[i] > 0:
            Csol.append(i)
    wb = Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)
    clash_sheet = wb.create_sheet(title='Clash Report')
    exam_slot = wb.create_sheet(title='Exam Slot report')
    seating_sheet = wb.create_sheet(title='Seating Report')

    clash_report(X_sol, clash_sheet)
    exam_slot_report(Esol, Z_sol, exam_slot)
    seating_report(C_sol, Csol, seating_sheet)

    save_path = "timetable_report.xlsx"
    wb.save(save_path)



In [ ]:
# extract E C M X values for all best 3 solution counts and print them

def extract_values(model):
    E_sol = model.getAttr('x', E)
    C_sol = model.getAttr('x', C)
    M_sol = model.getAttr('x', M)
    X_sol = model.getAttr('x', X)
    Z_sol = model.getAttr('x', Z)
    return E_sol, C_sol, M_sol, X_sol, Z_sol
# make a 3 pd dataframe for the sol

# E_sol C_sol M_sol X_sol are sets, i want to crerate a dataframe with columns E_sol, C_sol, M_sol, X_sol
def get_bestmax3_solutions(model):
    for i in range(min(1, model.SolCount)):
        s = {}
        model.setParam('SolutionNumber', i)
        E_sol, C_sol, M_sol, X_sol, Z_sol = extract_values(model)
        s['E_sol'] = E_sol
        s['C_sol'] = C_sol
        s['M_sol'] = M_sol
        s['X_sol'] = X_sol
        s['Z_sol'] = Z_sol
        make_report(s)

get_bestmax3_solutions(tt)


In [ ]:
E_sol, C_sol, M_sol, X_sol, Z_sol = extract_values(tt)

solution = []
for i in E_sol:
    if E_sol[i] > 0:
        solution.append(i)

GurobiError: Unable to retrieve attribute 'x'

In [ ]:
solution_rooms=[]
for i in C_sol:
    if C_sol[i] > 0:
        solution_rooms.append(i)

In [ ]:
schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri","Sat"]

print("EXAM TIMETABLE")
print("=" * 50)
for day in weekdays:
    if day in schedule_by_day:
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        for entry in sorted_entries:
            del entry["start"]
        print(f"\n{day}")
        print("-" * 50)
        print(tabulate(sorted_entries, headers="keys", tablefmt="grid"))

In [ ]:
schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]


# ----- Save timetable to CSV/Excel -----
rows = []
for day in weekdays:
    if day in schedule_by_day:
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        for entry in sorted_entries:
            temp_entry = dict(entry)
            temp_entry.pop("start", None)
            temp_entry["Day"] = day
            rows.append(temp_entry)

df = pd.DataFrame(rows)
df.to_csv("exam_timetable.csv", index=False)
df.to_excel("exam_timetable.xlsx", index=False)
print("\nCSV and Excel files have been saved as exam_timetable.csv and exam_timetable.xlsx")

# ----- Save timetable to PDF -----
styles = getSampleStyleSheet()
elements = []
doc = SimpleDocTemplate("exam_timetable.pdf", pagesize=LETTER)
title = Paragraph("EXAM TIMETABLE", styles["Title"])
elements.append(title)
elements.append(Spacer(1, 12))

for day in weekdays:
    if day in schedule_by_day:
        day_header = Paragraph(f"<b>{day}</b>", styles["Heading2"])
        elements.append(day_header)
        elements.append(Spacer(1, 6))
        # Prepare table data.
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        # Use shallow copies so we don't remove the "start" key from the original.
        pdf_entries = [dict(entry) for entry in sorted_entries]
        for entry in pdf_entries:
            entry.pop("start", None)
        if pdf_entries:
            headers = list(pdf_entries[0].keys())
            table_data = [headers]
            for entry in pdf_entries:
                row_data = [entry[h] for h in headers]
                table_data.append(row_data)
            t = Table(table_data, hAlign="LEFT")
            t.setStyle(TableStyle([
                ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
                ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
                ("ALIGN", (0, 0), (-1, -1), "LEFT"),
                ("GRID", (0,0), (-1,-1), 1, colors.black),
                ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold")
            ]))
            elements.append(t)
            elements.append(Spacer(1, 12))

doc.build(elements)
print("PDF file has been saved as exam_timetable.pdf")

In [ ]:
from collections import defaultdict
from tabulate import tabulate

# Turn interactive mode off.
plt.ioff()

# Helper: convert a time string "HH:MM" to minutes since midnight.
def time_to_minutes(timestr):
    h, m = map(int, timestr.split(':'))
    return h * 60 + m

# --- Build the textual timetable (from your provided code) ---
schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]  # used for sorting only
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]

# --- Create the visualization for all days on one page ---
days_with_exams = [day for day in weekdays if day in schedule_by_day]
n_days = len(days_with_exams)

fig, axes = plt.subplots(n_days, 1, figsize=(10, 2.5 * n_days), sharex=True)
if n_days == 1:
    axes = [axes]

# Colors for exam blocks.
colors = plt.cm.tab10.colors

for ax, day in zip(axes, days_with_exams):
    exams = sorted(schedule_by_day[day], key=lambda x: time_to_minutes(x["Time"].split(" - ")[0]))
    for idx, exam in enumerate(exams):
        start_str, end_str = exam["Time"].split(" - ")
        start_min = time_to_minutes(start_str)
        end_min = time_to_minutes(end_str)
        duration = end_min - start_min

        rect = mpatches.Rectangle((start_min, idx - 0.4), duration, 0.8,
                                  color=colors[idx % len(colors)], alpha=0.8)
        ax.add_patch(rect)
        label = f"{exam['Course']}\n(Str: {exam['Strength']})"
        ax.text(start_min + duration/2, idx, label, ha="center", va="center", fontsize=9, color="black")
    
    ax.set_ylim(-1, len(exams))
    ax.set_yticks(range(len(exams)))
    ytick_labels = [exam["Rooms"] + " | Str: " + str(exam["Strength"]) for exam in exams]
    ax.set_yticklabels(ytick_labels)
    ax.set_title(day)
    ax.grid(True, axis="x", linestyle="--", alpha=0.6)

all_start = min(time_to_minutes(exam["Time"].split(" - ")[0])
                for day in days_with_exams 
                for exam in schedule_by_day[day])
all_end = max(time_to_minutes(exam["Time"].split(" - ")[1])
              for day in days_with_exams 
              for exam in schedule_by_day[day])
ax_pad = 30
plt.xlim(all_start - ax_pad, all_end + ax_pad)

xticks = list(range((all_start - ax_pad) // 60 * 60, (all_end + ax_pad) + 1, 60))
xtick_labels = [f"{t//60:02d}:{t % 60:02d}" for t in xticks]
plt.xticks(xticks, xtick_labels, rotation=45)
plt.xlabel("Time")

plt.savefig("exam_timetable_plot.pdf")
plt.close(fig)  # close the figure to avoid output

print("Visualization saved as exam_timetable_plot.pdf")

# --- Create separate visualization for each day (wider plots) ---
for day in days_with_exams:
    exams = sorted(schedule_by_day[day], key=lambda x: time_to_minutes(x["Time"].split(" - ")[0]))
    n_exams = len(exams)
    # Increase the width to 16 inches (adjust as needed) and scale the height based on number of exams.
    fig, ax = plt.subplots(figsize=(16, max(4, n_exams * 1.5)))
    
    colors = plt.cm.tab10.colors

    for idx, exam in enumerate(exams):
        start_str, end_str = exam["Time"].split(" - ")
        start_min = time_to_minutes(start_str)
        end_min = time_to_minutes(end_str)
        duration = end_min - start_min

        rect = mpatches.Rectangle((start_min, idx - 0.4), duration, 0.8,
                                  color=colors[idx % len(colors)], alpha=0.8)
        ax.add_patch(rect)
        label = f"{exam['Course']}\n(Str: {exam['Strength']})"
        ax.text(start_min + duration/2, idx, label, ha="center", va="center", fontsize=9, color="black")
    
    ax.set_ylim(-1, n_exams)
    ax.set_yticks(range(n_exams))
    ytick_labels = [exam["Rooms"] + " | Str: " + str(exam["Strength"]) for exam in exams]
    ax.set_yticklabels(ytick_labels)
    ax.set_title(day)
    ax.grid(True, axis="x", linestyle="--", alpha=0.6)

    all_start = min(time_to_minutes(exam["Time"].split(" - ")[0]) for exam in exams)
    all_end = max(time_to_minutes(exam["Time"].split(" - ")[1]) for exam in exams)
    pad = 30
    ax.set_xlim(all_start - pad, all_end + pad)
    xticks = list(range((all_start - pad) // 60 * 60, all_end + pad + 1, 60))
    xtick_labels = [f"{t//60:02d}:{t % 60:02d}" for t in xticks]
    ax.set_xticks(xticks)
    ax.set_xticklabels(xtick_labels, rotation=45)
    ax.set_xlabel("Time")
    
    fig.savefig(f"exam_timetable_plot_{day}.pdf")
    plt.close(fig)  # close the figure

print("Separate visualizations saved as exam_timetable_plot_{day}.pdf files for each day")

In [ ]:

def merge_pdfs(pdf_list, output_filename):
    merger = PyPDF2.PdfMerger()
    
    for pdf in pdf_list:
        merger.append(pdf)

    merger.write(output_filename)
    merger.close()
    print(f"Merged PDF saved as {output_filename}")

# List of input PDFs (ensure correct file paths)
pdf_files = [
    "exam_timetable_plot_Mon.pdf",
    "exam_timetable_plot_Tue.pdf",
    "exam_timetable_plot_Wed.pdf",
    "exam_timetable_plot_Thu.pdf",
    "exam_timetable_plot_Fri.pdf",
    "exam_timetable_plot_Sat.pdf"
]
output_pdf = "exam_timetable_plot.pdf"



merge_pdfs(pdf_files, output_pdf)

# Delete all the pdf_files
for pdf in pdf_files:
    os.remove(pdf)


In [ ]:
## Verification of the solution

# 1 - No student should have two exams at the same time

def time_to_minutes(time_str):
    """Converts a HH:MM string to minutes since midnight."""
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

# Build a dictionary that maps each course (c) to its one exam slot details.
# Assume each course appears only once in the solution.
course_slots = {}
for c, b, e, d in solution:
    # Save day and exam start/end times (converted to minutes) for each course
    day = data.days[d]
    start = time_to_minutes(data.times[b])
    end   = time_to_minutes(data.times[e])
    course_slots[c] = (day, start, end)

# Check for any student conflicts between every pair of distinct courses.
clash_found = False
course_indices = list(course_slots.keys())
for i in range(len(course_indices)):
    c1 = course_indices[i]
    day1, start1, end1 = course_slots[c1]
    for j in range(i + 1, len(course_indices)):
        c2 = course_indices[j]
        day2, start2, end2 = course_slots[c2]
        # Check if courses are on the same day and if their times overlap:
        if day1 == day2 and (start1 < end2 and start2 < end1):
            # Check intersection of enrolled students; get_intersection_of_courses returns set of common student IDs.
            common_students = set(data.course_student[c1]) & set(data.course_student[c2])
            if common_students:
                clash_found = True
                print(f"Clash found between Course {data.course_id_code[c1]} and Course {data.course_id_code[c2]}")
                print("Common student(s):", common_students)
                print(f"--> {data.course_id_code[c1]}: {data.times[b]} - {data.times[e]} ;",  # later you can format times properly
                      f"{data.course_id_code[c2]}: {data.times[b]} - {data.times[e]}\n")

if not clash_found:
    print("No student clashes found!")


# 2 - The total number of students writing an exam in a room should not exceed the room capacity


room_violations = False

# Process each room separately
for r in range(data.n_rooms):
    # Build a schedule per day for room r.
    room_schedule = defaultdict(list)
    for c, b, e, d in solution:
        allocated = C_sol[c, r]
        if allocated > 0:
            day = data.days[d]
            start = time_to_minutes(data.times[b])
            end = time_to_minutes(data.times[e])
            course_code = data.course_id_code[c]
            room_schedule[day].append((start, end, allocated, course_code))
    
    # For each day, process the exam intervals for room r.
    for day, intervals in room_schedule.items():
        events = []
        for start, end, alloc, course in intervals:
            # When exam starts, add the number of students;
            # when exam ends, subtract that number.
            events.append((start, alloc))
            events.append((end, -alloc))
        events.sort(key=lambda x: x[0])
        
        current_alloc = 0
        for time, change in events:
            current_alloc += change
            if current_alloc > data.get_room_capacity(r):
                room_violations = True
                print(f"Room capacity violation on {day} in room {data.get_room_num(r)}:")
                print(f"   Room capacity: {data.get_room_capacity[r]}, allocated: {current_alloc}")
                break  # Report once per day per room

if not room_violations:
    print("No room capacity violations found!")


In [ ]:
for s in range (data.n_students):
    l=[]
    course_list=data.get_courses_student(s)
    for c,b,e,d in solution:
        if E_sol[c,b,e,d] == 1 and c in course_list:
            l.append((data.course_id_code[c], data.times[b], data.times[e], data.days[d]))
    
    print(f"Student {s+1} has exams for courses {l}")

In [ ]:
for s in range(data.n_students):
    print(f"Student {s+1} has exam on: " , [data.days[d] for c,b,e,d in solution if E_sol[c,b,e,d] == 1 and c in data.get_courses_student(s)])

In [ ]:
# print how many courses have got the preferred slots
for c in range(data.n_courses):
    print(f"Course {data.course_id_code[c]} has got the preferred slots: ", [data.times[b] for b,e,d in data.get_preferred_slots(c) if E_sol[c,b,e,d] == 1])

In [ ]:
for c1, c2 in data.intersection_pairs:
    clash = 0
    for d in range(data.n_days):
        clash += X_sol[c1, c2, d]
        if(X_sol[c1, c2, d] > 0):
            print(data.course_id_code[c1], data.course_id_code[c2], d)
            
    print(data.course_id_code[c1], data.course_id_code[c2], clash>0)

In [ ]:
# print all X(c1, c2, d) that are non zeros
for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        if X_sol[c1, c2, d] == 0:
            print(data.course_id_code[c1], data.course_id_code[c2], d)
        # print sum of E[c1, *, *, d] + E[c2, *, *, d] for that d
        Ec1d = 0
        for b, e in exam_spans[c1]:
            Ec1d += E_sol[c1, b, e, d]
        Ec2d = 0
        for b, e in exam_spans[c2]:
            Ec2d += E_sol[c2, b, e, d]
        print(data.course_id_code[c1], data.course_id_code[c2], d, Ec1d, Ec2d)
        print(X_sol[c1, c2, d])

In [ ]:
print(solution)

In [ ]:
count = 0
for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        se=0
        for b, e in exam_spans[c1]:
            se += E_sol[c1, b, e, d] 
        for b, e in exam_spans[c2]:
            se += E_sol[c2, b, e, d]
        if(se > 1):
            count+=1
            print(f'{len(data.get_intersection_of_courses(c1,c2))},X: {X_sol[c1, c2, d]}')

print(count)

In [ ]:
len(data.intersection_pairs)

In [ ]:
# # Cleaning Up
# os.remove("course_master.csv")
# os.remove("enrollment_master.csv")
# os.remove("exam_choice.csv")
# os.remove("exam_master.csv")

In [ ]:
# # Cleaning Up after viewing the report

# os.remove("students.csv")
# os.remove("timetable_report.xlsx")
# os.remove("exam_timetable.xlsx")
# os.remove("exam_timetable_plot.pdf")
# os.remove("exam_timetable.pdf")
# os.remove("exam_timetable.csv")